<a href="https://colab.research.google.com/github/bbanzai88/Book_Writing_Crew/blob/main/Category_Theory_Cancer_Analysisv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip -q uninstall -y opencv-python opencv-contrib-python opencv-python-headless thinc spacy


In [3]:
!pip -q install "numpy==1.26.4"


In [4]:
!pip -q install "thinc==8.2.5" "spacy==3.7.4"
!pip -q install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.2/920.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 3.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.41.0 requires typer<1.0,>=0.12; sys_platform != "emscripten", but you have typer 0.9.4 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 57.2 MB/s eta 0:00:00


In [ ]:
import os, signal; os.kill(os.getpid(), signal.SIGKILL)


In [1]:
!pip -q install "requests==2.32.3" "pdfminer.six==20231228" \
                "lxml" "tqdm" "pandas==2.2.2" "networkx==3.2.1" \
                "pyvis==0.3.2" "feedparser"


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requi

In [2]:
import spacy, numpy
print("spaCy:", spacy.__version__, "| NumPy:", numpy.__version__)
spacy.load("en_core_web_sm"); print("spaCy model OK")


spaCy: 3.7.4 | NumPy: 1.26.4
spaCy model OK


In [11]:
# ============================================
# ONE-CELL E2E: PubMed + xRxiv → Filter → Triples (Ollama) → KG → Viz
# ============================================
# --- Config (edit these) ---
DATE_FROM = "2013-11-01"         # YYYY-MM-DD
DATE_TO   = None                 # None = today
XRXIV_WINDOW_DAYS = 365          # number of days back for biorxiv/medrxiv/chemrxiv
SEARCH_TERMS = [
    "(cancer OR carcinoma OR neoplasm OR oncology OR tumor)",
    "(mitochondria OR mitochondrial OR metabolism OR metabolic)"
]
KEYWORDS = {"cancer","neoplasm","carcinoma","oncology","tumor","tumour",
            "mitochondria","mitochondrial","metabolism","metabolic"}
FULLTEXT = False                  # keep False unless you really want PDFs (slow)
MAX_PAPERS = 5000                 # cap processed records (None = all; keep modest first run)
OLLAMA_MODEL = "deepseek-r1:1.5b" # small + quick; you can change later

# --- Paths ---
import os, signal, sys, textwrap, json, gzip, io, time, math, random, itertools, re
from pathlib import Path
from datetime import datetime, date, timedelta
from collections import Counter, defaultdict

BASE = Path("/content")
RAW  = BASE/"raw_corpus"; RAW.mkdir(exist_ok=True)
FIL  = BASE/"filtered_corpus"; FIL.mkdir(exist_ok=True)
OUT  = BASE/"kg_out"; OUT.mkdir(exist_ok=True)

# =============================================================================
# 0) Dependencies (pin to avoid Colab binary mismatches)
# =============================================================================
print("Installing deps…")
!pip install -q --force-reinstall "numpy==1.26.4" "spacy==3.7.4" "thinc==8.2.5" \
    "requests==2.32.3" "lxml" "tqdm" "pandas==2.2.2" "networkx==3.2.1" \
    "pyvis==0.3.2" "feedparser"
!pip install -q https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl

# =============================================================================
# 1) Imports + small helpers
# =============================================================================
import spacy, requests, pandas as pd, networkx as nx
from urllib.parse import urlencode, quote
from tqdm import tqdm
from IPython.display import HTML, display
from pyvis.network import Network

def today_iso(): return date.today().isoformat()
if DATE_TO is None: DATE_TO = today_iso()

def as_iso(d):
    if not d: return None
    try: return datetime.fromisoformat(str(d).replace("Z","+00:00")).date().isoformat()
    except:
        s=str(d)
        return s.split("T",1)[0] if "T" in s else s

def within_window(pubd, d0, d1):
    if not pubd: return False
    return (d0 <= pubd <= d1)

def gz_write(path: Path, records):
    n=0
    with gzip.open(path, "wt", encoding="utf-8") as f:
        for r in records:
            f.write(json.dumps(r, ensure_ascii=False)+"\n"); n+=1
    return n

def gz_iter(path: Path):
    with gzip.open(path, "rt", encoding="utf-8", errors="ignore") as f:
        for line in f:
            try: yield json.loads(line)
            except: continue

# =============================================================================
# 2) Start Ollama locally in Colab and pull model
# =============================================================================
print("Setting up Ollama…")
import threading, subprocess

# Clean conflicting envs
for v in ["OPENAI_API_KEY","LITELLM_PROVIDER","LITELLM_MODEL","LITELLM_BASE_URL"]:
    os.environ.pop(v, None)
os.environ["OLLAMA_HOST"]    = "0.0.0.0:11434"
os.environ["OLLAMA_ORIGINS"] = "*"

# Install Ollama
!curl -fsSL https://ollama.com/install.sh -o install.sh
!bash install.sh >/dev/null 2>&1 || true

def _serve_ollama():
    subprocess.Popen(["ollama","serve"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
threading.Thread(target=_serve_ollama, daemon=True).start()
time.sleep(8)

try:
    ok = requests.get("http://127.0.0.1:11434").ok
    print("Ollama API:", "OK" if ok else "not responding")
except Exception as e:
    print("Ollama check failed:", e)

print("Pulling model:", OLLAMA_MODEL)
!ollama pull {OLLAMA_MODEL} >/dev/null 2>&1 || true

# =============================================================================
# 3) spaCy lightweight pipeline (disease-ish mentions) + helpers
# =============================================================================
nlp = spacy.load("en_core_web_sm")
if "sentencizer" not in nlp.pipe_names:
    nlp.add_pipe("sentencizer")

DISEASE_TERMS = {
    "cancer","malignancy","neoplasm","tumor","tumour","carcinoma","sarcoma","lymphoma",
    "leukemia","melanoma","glioma","blastoma","myeloma","metastasis","metastases",
    "apoptosis","proliferation","metabolic reprogramming","oxidative stress","p53 pathway",
    "mitochondrial dysfunction","ros","dna damage","glycolysis","warburg effect"
}
DISEASE_RE = re.compile(
    r"\b(?:[A-Za-z\-]+)(?:carcinoma|sarcoma|lymphoma|melanoma|glioma|blastoma|myeloma|leukemia|tumou?r|neoplasm|metastas(?:is|es))s?\b",
    flags=re.I
)

from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
matcher.add("DISEASE_GAZ", [nlp.make_doc(t) for t in sorted(DISEASE_TERMS)])

def disease_spans(span_or_doc):
    if isinstance(span_or_doc, Span):
        sent=span_or_doc; doc=sent.doc; s_tok, e_tok = sent.start, sent.end; s_char = sent.start_char
    else:
        sent=None; doc=span_or_doc; s_tok, e_tok = 0, len(doc); s_char=0
    spans=[]
    for _, a, b in matcher(doc):
        if a>=s_tok and b<=e_tok:
            spans.append(Span(doc, a, b, label="DISEASE"))
    taken={(sp.start, sp.end) for sp in spans}
    txt = (sent.text if sent is not None else doc.text)
    for m in DISEASE_RE.finditer(txt):
        cs = (s_char + m.start()) if sent is not None else m.start()
        ce = (s_char + m.end())   if sent is not None else m.end()
        sp = doc.char_span(cs, ce, alignment_mode="expand")
        if sp and sp.start>=s_tok and sp.end<=e_tok and (sp.start, sp.end) not in taken:
            spans.append(Span(doc, sp.start, sp.end, label="DISEASE"))
    spans.sort(key=lambda s:(s.start, -len(s)))
    merged=[]
    for s in spans:
        if not merged or s.start>=merged[-1].end: merged.append(s)
        else:
            prev=merged[-1]
            if (s.end-s.start)>(prev.end-prev.start): merged[-1]=s
    return merged

def canon(x:str)->str: return re.sub(r"\s+"," ", (x or "").strip()).lower()

# =============================================================================
# 4) PubMed E-utilities (robust JSON with retry)
# =============================================================================
# ---- Robust PubMed E-utilities (drop-in replacement) ----
import requests, re, time
from urllib.parse import urlencode

NCBI_EMAIL = "you@example.com"           # put your email (recommended by NCBI)
NCBI_TOOL  = "colab-kg"
NCBI_API_KEY = os.environ.get("NCBI_API_KEY", "")  # optional: set in env for higher limits

S = requests.Session()
S.trust_env = False
S.headers.update({"User-Agent": f"{NCBI_TOOL}/1.0", "Accept-Encoding":"identity"})

def _pm_get(url, params, tries=5, pause=1.5):
    # add standard params
    params = dict(params)  # copy
    params.setdefault("tool", NCBI_TOOL)
    params.setdefault("email", NCBI_EMAIL)
    if NCBI_API_KEY: params["api_key"] = NCBI_API_KEY

    last_txt = ""
    for i in range(tries):
        r = S.get(url, params=params, timeout=45)
        # 200 but might be HTML; keep text for debug if needed
        last_txt = r.text[:400]
        if r.status_code == 200:
            return r
        # simple backoff on 429/5xx
        if r.status_code in (429, 500, 502, 503, 504):
            time.sleep(pause * (i+1))
            continue
        r.raise_for_status()
    # final raise with hint
    raise RuntimeError(f"PubMed request failed after retries. Sample response: {last_txt}")

def pubmed_search(term, dfrom, dto):
    """Return list of PMIDs for term in date window (publication date). Robust to non-JSON ESearch replies."""
    term = term.replace("\n", " ").strip()
    base = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    params = {
        "db":"pubmed",
        "term": term,
        "retmode": "json",
        "retmax": 0,
        "usehistory": "y",
        "mindate": dfrom,
        "maxdate": dto,
        "datetype": "pdat",
    }
    r = _pm_get(base, params)
    # Try JSON first; if that fails, fallback to XML parse
    try:
        data = r.json()
        count = int(data["esearchresult"]["count"])
        if count == 0:
            return []
        webenv = data["esearchresult"]["webenv"]
        query_key = data["esearchresult"]["querykey"]
        ids = []
        for retstart in range(0, count, 10000):
            rr = _pm_get(base, {
                "db":"pubmed", "retmode":"json",
                "query_key": query_key, "WebEnv": webenv,
                "retmax": min(10000, count-retstart), "retstart": retstart
            }).json()
            ids.extend(rr["esearchresult"].get("idlist", []))
        return ids
    except Exception:
        # Fallback: re-run in XML and scrape idlist (rare, but saves the run)
        rx = _pm_get(base, {**params, "retmode":"xml"})
        xml = rx.text
        # quick count check (optional)
        # extract IDs if present (when retmax>0); since we had retmax=0 above, we redo with retmax large
        rx2 = _pm_get(base, {**params, "retmode":"xml", "retmax":100000})
        xml2 = rx2.text
        ids = re.findall(r"<Id>(\d+)</Id>", xml2)
        return ids

def pubmed_fetch(pmids):
    """Fetch minimal fields from PubMed efetch (XML)."""
    if not pmids:
        return []
    base = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
    out = []
    for i in range(0, len(pmids), 200):
        chunk = pmids[i:i+200]
        r = _pm_get(base, {"db":"pubmed", "id":",".join(chunk), "retmode":"xml"})
        txt = r.text
        for art in re.findall(r"<PubmedArticle[\s\S]*?</PubmedArticle>", txt):
            def one(tag):
                m = re.search(fr"<{tag}[^>]*>([\s\S]*?)</{tag}>", art)
                return (m.group(1).strip() if m else "")
            title = one("ArticleTitle")
            ab = re.search(r"<AbstractText[^>]*>([\s\S]*?)</AbstractText>", art)
            ab = ab.group(1).strip() if ab else ""
            pm = one("PMID")
            doi_m = re.search(r"<ELocationID[^>]*EIdType=\"doi\"[^>]*>(.*?)</ELocationID>", art)
            doi = doi_m.group(1).strip() if doi_m else ""
            year = re.search(r"<PubDate>[\s\S]*?<Year>(\d{4})</Year>", art)
            month= re.search(r"<PubDate>[\s\S]*?<Month>(\w+)</Month>", art)
            day  = re.search(r"<PubDate>[\s\S]*?<Day>(\d{1,2})</Day>", art)
            yyyy = year.group(1) if year else "1900"
            mm   = month.group(1) if month else "01"
            dd   = day.group(1) if day else "01"
            try:
                from calendar import month_abbr
                if not mm.isdigit():
                    mm = str(list(month_abbr).index(mm[:3].title())).zfill(2)
                dd = dd.zfill(2)
            except Exception:
                mm="01"; dd="01"
            pubd = f"{yyyy}-{mm}-{dd}"
            out.append({
                "server":"pubmed", "pmid":pm, "doi":doi,
                "title":title, "abstract":ab, "published_date":as_iso(pubd),
                "url":f"https://pubmed.ncbi.nlm.nih.gov/{pm}/"
            })
        # be polite
        time.sleep(0.34 if NCBI_API_KEY else 1.0)
    return out

# =============================================================================
# 5) xRxiv fetchers (windowed, resilient ChemRxiv via Figshare)
# =============================================================================
def biorxiv_fetch(server, dfrom, dto):
    base=f"https://api.biorxiv.org/details/{server}/{dfrom}/{dto}"
    cursor=0; seen=set()
    while True:
        url=f"{base}/{cursor}"
        r=S.get(url, timeout=30); r.raise_for_status()
        data=r.json(); coll=data.get("collection",[])
        if not coll: break
        for rec in coll: yield rec
        nxt=data.get("next_cursor")
        if nxt is None or nxt in seen: break
        seen.add(nxt); cursor=nxt; time.sleep(0.1)

def chemrxiv_fetch(dfrom, dto):
    def _iso_date_only(s):
        try: return datetime.fromisoformat(s.replace("Z","+00:00")).date().isoformat()
        except: return s.split("T",1)[0] if "T" in s else s
    base_urls=[
        "https://api.figshare.com/v2/articles/search",
        "https://api.figshare.com/v2/articles/search/"
    ]
    total=0; seen_ids=set()
    for base in base_urls:
        strategy="A"; page=1
        while True:
            params={"search_for":"ChemRxiv","page":page,"page_size":200,"order":"published_date","order_direction":"desc"}
            if strategy in ("A","B"):
                params["published_since"]=dfrom; params["published_until"]=dto
            if strategy=="A": params["institution"]="ChemRxiv"
            r=S.get(base, params=params, timeout=30)
            if r.status_code in (400,404):
                if strategy=="A": strategy="B"; page=1; continue
                elif strategy=="B": strategy="C"; page=1; continue
                else: break
            r.raise_for_status(); items=r.json()
            if not items:
                if page==1:
                    if strategy=="A": strategy="B"; continue
                    elif strategy=="B": strategy="C"; continue
                break
            wrote=0
            for it in items:
                art_id=it.get("id");
                if art_id in seen_ids: continue
                seen_ids.add(art_id)
                pubd=_iso_date_only(it.get("published_date","") or (it.get("timeline") or {}).get("published",""))
                if strategy=="C" and not within_window(pubd, dfrom, dto): continue
                yield {
                    "server":"chemrxiv","id":art_id,
                    "title":it.get("title","") or "",
                    "abstract":it.get("description","") or "",
                    "doi":it.get("doi") or it.get("doi_url"),
                    "published_date":as_iso(pubd),
                    "url":it.get("url_public_html") or it.get("url")
                }
                wrote+=1; total+=1
            if wrote==0:
                if strategy=="A": strategy="B"; page=1
                elif strategy=="B": strategy="C"; page=1
                else: break
            else:
                page+=1; time.sleep(0.12)
        if total>0: break

# =============================================================================
# 6) Crawl → Filter → Combine
# =============================================================================
print(f"\nCrawling PubMed for {DATE_FROM} → {DATE_TO}")
pm_all=[]
for term in SEARCH_TERMS:
    ids=pubmed_search(f"({term})", DATE_FROM, DATE_TO)
    print(f"PubMed ids for [{term}]: {len(ids)}")
    pm_all.extend(ids)
pm_all=list(dict.fromkeys(pm_all))
if pm_all:
    pubmed_records=pubmed_fetch(pm_all)
    n=gz_write(RAW/"pubmed.jsonl.gz", pubmed_records)
    print("Wrote PubMed:", n)
else:
    print("⚠️ No PubMed matches")

print("\nCrawling xRxiv (windowed)…")
dfrom = (datetime.fromisoformat(DATE_TO).date() - timedelta(days=XRXIV_WINDOW_DAYS)).isoformat()
dto   = DATE_TO
bx=list(biorxiv_fetch("biorxiv", dfrom, dto)); mx=list(biorxiv_fetch("medrxiv", dfrom, dto)); cx=list(chemrxiv_fetch(dfrom, dto))
gz_write(RAW/"biorxiv.jsonl.gz", bx)
gz_write(RAW/"medrxiv.jsonl.gz", mx)
gz_write(RAW/"chemrxiv.jsonl.gz", cx)
print(f"Window: {dfrom} → {dto} | bioRxiv:{len(bx)} medRxiv:{len(mx)} ChemRxiv:{len(cx)}")

print("\nFiltering + de-dup…")
KEYS = {k.lower() for k in KEYWORDS}
def text_for(rec):
    return ((rec.get("title") or "") + "\n" + (rec.get("abstract") or rec.get("description") or "")).lower()
def passes(rec):
    t=text_for(rec)
    return any(k in t for k in KEYS)
def normalize(rec):
    r=dict(rec)
    r["published_date"]=as_iso(r.get("published_date") or r.get("date"))
    return r

seen=set(); filtered=[]
for p in [RAW/"pubmed.jsonl.gz", RAW/"biorxiv.jsonl.gz", RAW/"medrxiv.jsonl.gz", RAW/"chemrxiv.jsonl.gz"]:
    if not p.exists(): continue
    for rec in gz_iter(p):
        r=normalize(rec)
        if not r.get("title") and not r.get("abstract"): continue
        pubd=r.get("published_date") or "1900-01-01"
        if not within_window(pubd, DATE_FROM, DATE_TO): continue
        if not passes(r): continue
        k=( (r.get("doi") or "").lower().strip(),
            (r.get("pmid") or r.get("id") or r.get("url") or "").lower().strip(),
            pubd )
        if k in seen: continue
        seen.add(k); filtered.append(r)

FILLED = FIL/"filtered_all.jsonl.gz"
gz_write(FILLED, filtered)
print(f"Filtered kept: {len(filtered)} → {FILLED}")

# =============================================================================
# 7) Triples: rule-based + (optional) Ollama
# =============================================================================
def ollama_triples(sentence: str, concepts):
    if not concepts or len(concepts)<2: return []
    try:
        payload={"model": OLLAMA_MODEL,
                 "prompt": ("Extract a JSON array of triples with keys subject, relation, object. "
                            "Subject and object MUST be chosen from exactly this set:\n"
                            f"{json.dumps(concepts, ensure_ascii=False)}\n\n"
                            f"Sentence: {json.dumps(sentence)}\nReturn only JSON."),
                 "temperature":0.0, "max_tokens":256}
        r=S.post("http://127.0.0.1:11434/completions", json=payload, timeout=60)
        r.raise_for_status()
        txt=r.json().get("choices",[{}])[0].get("text","")
        txt=re.sub(r"<think>.*?</think>", "", txt, flags=re.S)
        m=re.search(r"\[[\s\S]*\]", txt)
        if not m: return []
        arr=json.loads(m.group(0))
        out=[]
        for d in arr:
            s=d.get("subject","").strip(); o=d.get("object","").strip(); rel=d.get("relation","").strip()
            if s and o and rel: out.append({"subject":s,"relation":rel,"object":o,"method":"llm"})
        return out
    except Exception:
        return []

def lca_token(e1: Span, e2: Span):
    anc1={t for t in e1.root.ancestors}
    for t in e2.root.ancestors:
        if t in anc1: return t
    return e1.sent.root

def rule_triples(sent: Span, ents):
    out=[]
    from itertools import combinations
    for a,b in combinations(ents,2):
        lca=lca_token(a,b)
        if lca.pos_ in ("VERB","AUX"):
            out.append({"subject":a.text,"relation":lca.lemma_,"object":b.text,"method":"rule"})
            out.append({"subject":b.text,"relation":lca.lemma_,"object":a.text,"method":"rule"})
    return out

print("\nExtracting triples (this pass caps papers for speed)…")
triples_gz = OUT/"triples.jsonl.gz"
G = nx.DiGraph()
first_label = {}

tw = gzip.open(triples_gz, "wt", encoding="utf-8")
processed=0
for rec in tqdm(filtered[: (MAX_PAPERS or len(filtered)) ]):
    text = ((rec.get("title") or "") + "\n\n" + (rec.get("abstract") or rec.get("description") or "")).strip()
    if not text: continue
    doc=nlp(text)
    pid = str(rec.get("pmid") or rec.get("doi") or rec.get("id") or rec.get("url") or processed)

    for sent in doc.sents:
        ents=disease_spans(sent)
        if len(ents)<2: continue
        concepts=list(dict.fromkeys([e.text.strip() for e in ents]))[:12]

        r1=rule_triples(sent, ents)
        r2=ollama_triples(sent.text, concepts)

        for t in itertools.chain(r1,r2):
            s_raw=t["subject"]; o_raw=t["object"]; rel_raw=t["relation"]
            if not s_raw or not o_raw or not rel_raw: continue
            s=canon(s_raw); o=canon(o_raw); rel=canon(rel_raw)
            if s==o: continue

            first_label.setdefault(("node",s), s_raw)
            first_label.setdefault(("node",o), o_raw)
            first_label.setdefault(("rel",rel), rel_raw)

            G.add_node(s, label=first_label[("node",s)])
            G.add_node(o, label=first_label[("node",o)])
            if G.has_edge(s,o):
                G[s][o]["weight"] += 1
                G[s][o].setdefault("relations", Counter())
                G[s][o]["relations"][rel] += 1
            else:
                G.add_edge(s,o, weight=1, relations=Counter({rel:1}))

            tw.write(json.dumps({
                "paper_id": pid, "subject": s_raw, "relation": rel_raw, "object": o_raw,
                "sentence": sent.text, "method": t.get("method","")
            }, ensure_ascii=False)+"\n")
    processed+=1
tw.close()
print(f"Graph so far: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
print("Triples →", triples_gz)

# =============================================================================
# 8) Category-theory flavored queries
# =============================================================================
def shared_precursors(G, outcome1, outcome2, max_depth=2):
    if outcome1 not in G or outcome2 not in G: return set()
    def anc_with_depth(t):
        out=set()
        for a in nx.ancestors(G,t):
            try:
                if nx.shortest_path_length(G, a, t) <= max_depth:
                    out.add(a)
            except Exception: pass
        return out
    return anc_with_depth(outcome1) & anc_with_depth(outcome2)

def max_product_paths(G, cutoff=4):
    strength={}
    for src in G.nodes:
        for tgt in G.nodes:
            if src==tgt: continue
            if not nx.has_path(G, src, tgt): continue
            best=0.0
            for path in nx.all_simple_paths(G, src, tgt, cutoff=cutoff):
                w=1.0
                for u,v in zip(path,path[1:]):
                    w *= float(G[u][v].get("weight",1.0))
                if w>best: best=w
            if best>0: strength[(src,tgt)] = best
    return strength

def find_contradictions(G):
    C=[]
    for a in G.nodes:
        for b in G.successors(a):
            for c in G.successors(a):
                if b==c: continue
                dset=set(G.successors(b)) & set(G.successors(c))
                for d in dset:
                    eff1=(tuple(G[a][b].get("relations",{}).keys()),
                          tuple(G[b][d].get("relations",{}).keys()))
                    eff2=(tuple(G[a][c].get("relations",{}).keys()),
                          tuple(G[c][d].get("relations",{}).keys()))
                    if eff1!=eff2:
                        C.append((a,b,c,d,eff1,eff2))
    return C

# quick demo runs
example_outcomes = [ ("p53 pathway","apoptosis"), ("mitochondrial dysfunction","apoptosis") ]
for o1,o2 in example_outcomes:
    inter = shared_precursors(G, canon(o1), canon(o2))
    print(f"Shared precursors for [{o1}] & [{o2}]:", len(inter))

strengths = max_product_paths(G, cutoff=4)
print("Top max-product links (up to 10):")
for (s,t),v in sorted(strengths.items(), key=lambda x:-x[1])[:10]:
    print(" ", G.nodes[s].get("label",s), "→", G.nodes[t].get("label",t), ":", v)

contradictions = find_contradictions(G)
print("Potential contradictions:", len(contradictions))

# =============================================================================
# 9) Viz (PyVis inline; prune to avoid hairball)
# =============================================================================
MIN_EDGE_FREQ=1
TOP_N_NODES=300
to_drop=[(u,v) for u,v,d in G.edges(data=True) if int(d.get("weight",1)) < MIN_EDGE_FREQ]
G.remove_edges_from(to_drop)
if G.number_of_nodes()>TOP_N_NODES:
    degs=dict(G.degree(weight="weight"))
    keep=set(sorted(degs, key=degs.get, reverse=True)[:TOP_N_NODES])
    G=G.subgraph(keep).copy()

def graphml_safe(G):
    H=G.copy()
    for u,v,d in H.edges(data=True):
        if "relations" in d:
            d["relations_str"]="; ".join(f"{first_label.get(('rel',k),k)}:{c}" for k,c in d["relations"].items())
            del d["relations"]
        d["weight"]=int(d.get("weight",1))
    for n,dat in H.nodes(data=True):
        if "label" in dat and not isinstance(dat["label"], (str,int,float,bool)):
            dat["label"]=str(dat["label"])
    return H

G_safe=graphml_safe(G)
edges_csv = OUT/"kg_edges.csv"
pd.DataFrame([{
    "subject":G_safe.nodes[u].get("label",u),
    "object": G_safe.nodes[v].get("label",v),
    "weight": int(d.get("weight",1)),
    "relations": d.get("relations_str","")
} for u,v,d in G_safe.edges(data=True)]).to_csv(edges_csv, index=False)
nx.write_graphml(G_safe, OUT/"kg.graphml")
nx.write_gexf(G_safe, OUT/"kg.gexf")

deg=dict(G_safe.degree(weight="weight"))
in_deg=dict(G_safe.in_degree(weight="weight"))
out_deg=dict(G_safe.out_degree(weight="weight"))
def size_fn(x): return max(8, 12*(1+math.log1p(x)))

net=Network(height="740px", width="100%", directed=True, notebook=True, cdn_resources="in_line")
net.barnes_hut(gravity=-20000, central_gravity=0.15, spring_length=160, spring_strength=0.01, damping=0.85)

for n,dat in G_safe.nodes(data=True):
    label=dat.get("label",n)
    w=deg.get(n,1.0); s=size_fn(w)
    outw=out_deg.get(n,0.0); inw=in_deg.get(n,0.0)
    if outw>inw*1.5: color="#4F8DD3"
    elif inw>outw*1.5: color="#D34F6A"
    else: color="#9AA0A6"
    net.add_node(n, label=label, title=f"deg(w)={w}", value=s, color=color)

for u,v,d in G_safe.edges(data=True):
    w=int(d.get("weight",1)); title=d.get("relations_str","")
    net.add_edge(u,v, value=w, title=f"{title} (w={w})")

html=net.generate_html()
html_path = OUT/"knowledge_graph.html"
with open(html_path, "w", encoding="utf-8") as f: f.write(html)
display(HTML(html))
print("\nSaved:", html_path, edges_csv, OUT/"kg.graphml", OUT/"kg.gexf", OUT/"triples.jsonl.gz")

# Also offer downloads (in Colab this pops up save dialogs)
try:
    from google.colab import files
    for p in [html_path, edges_csv, OUT/"kg.graphml", OUT/"kg.gexf", OUT/"triples.jsonl.gz"]:
        files.download(str(p))
except Exception:
    pass


Installing deps…
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 2.0.8 requires opencv-python-headless>=4.9.0.80, which is not installed.
dopamine-rl 4.1.2 requires opencv-python>=3.4.8.29, which is not installed.
albucore 0.0.24 requires opencv-python-headless>=4.9.0.80, which is not installed.
google-colab 1.0.0 requires ipython==7.34.0, but you have ipython 9.4.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and pla

100%|██████████| 5000/5000 [02:06<00:00, 39.49it/s]


Graph so far: 77 nodes, 560 edges
Triples → /content/kg_out/triples.jsonl.gz
Shared precursors for [p53 pathway] & [apoptosis]: 0
Shared precursors for [mitochondrial dysfunction] & [apoptosis]: 64
Top max-product links (up to 10):
  metastasis → apoptosis : 188245056.0
  apoptosis → metastasis : 188245056.0
  carcinoma → proliferation : 170733888.0
  proliferation → carcinoma : 170733888.0
  metastasis → malignancy : 137832045.0
  malignancy → metastasis : 137832045.0
  antitumor → carcinoma : 122714982.0
  carcinoma → antitumor : 122714982.0
  apoptosis → carcinoma : 117185664.0
  carcinoma → apoptosis : 117185664.0
Potential contradictions: 62002



Saved: /content/kg_out/knowledge_graph.html /content/kg_out/kg_edges.csv /content/kg_out/kg.graphml /content/kg_out/kg.gexf /content/kg_out/triples.jsonl.gz


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
# ================================
# Unified KG Pipeline (Colab, 1 cell)
# PubMed + xRxiv → filter → triples (Ollama + rules) → KG → CT-style analyses
# ================================
# ---- installs (compatible stack) ----
import os, sys, subprocess, time, json, gzip, io, math, re, itertools, random
from datetime import datetime, date, timedelta
print("Installing deps…")
get_ipython().run_line_magic("pip", "install -q --force-reinstall numpy==1.26.4 spacy==3.7.4 thinc==8.2.5")
get_ipython().run_line_magic("pip", "install -q pdfminer.six==20231228 lxml tqdm pandas==2.2.2 networkx==3.2.1 pyvis==0.3.2 feedparser requests==2.32.3")
get_ipython().run_line_magic("pip", "install -q https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl")

import requests, feedparser, pandas as pd, networkx as nx
from tqdm import tqdm
from urllib.parse import urlencode, quote
from pdfminer.high_level import extract_text as pdf_extract_text
from collections import Counter, defaultdict
from IPython.display import HTML, display
from pyvis.network import Network
import spacy
nlp = spacy.load("en_core_web_sm")
if "sentencizer" not in nlp.pipe_names:
    nlp.add_pipe("sentencizer")

# -------------------------
# CONFIG
# -------------------------
BASE = "/content/kg_out"
os.makedirs(BASE, exist_ok=True)
RAW = os.path.join(BASE, "raw"); os.makedirs(RAW, exist_ok=True)
FIL = os.path.join(BASE, "filtered"); os.makedirs(FIL, exist_ok=True)
OUT = os.path.join(BASE, "out"); os.makedirs(OUT, exist_ok=True)

# date window for crawling
DATE_FROM = (date.today() - timedelta(days=365*12)).isoformat()  # ~12y back
DATE_TO   = date.today().isoformat()

# xRxiv window (days back). Set None to stop; set e.g. 365
XRXIV_WINDOW_DAYS = 365

# keywords
KEYWORDS = {
    "cancer","neoplasm","carcinoma","oncology","tumor","tumour",
    "mitochondria","mitochondrial","metabolism","metabolic"
}

# priority entities to **force-detect** as nodes even if weak NER
PRIORITY_ENTITIES = {
    "p53 pathway","tp53","p53","apoptosis",
    "mitochondrial dysfunction","mitochondria","ros","oxidative stress",
    "glycolysis","warburg effect","proliferation","dna damage","cell cycle arrest"
}

# stop nodes (too generic → remove for analysis/viz; keep in raw if you want)
STOP_NODES = {"cancer","tumor","tumour","neoplasm","malignancy","carcinoma","metastasis"}

# aliases (coequalize)
ALIASES = {
    "p53":"tp53",
    "tumor protein p53":"tp53",
    "tp53 pathway":"p53 pathway",
}

# triples extraction
USE_FULLTEXT = False          # set True to try PDF fulltext (slow, best effort)
MAX_PAPERS_TRIPLES = 5000     # cap for speed; set None for all
OLLAMA_MODEL = "deepseek-r1:1.5b"

# visualization
TOP_N_NODES = 300
MIN_EDGE_COUNT = 1            # prune edges with count < this

# probability normalization (count -> p)
NORM_ALPHA = 10.0             # larger = slower saturation

# -------------------------
# Robust HTTP session
# -------------------------
sess = requests.Session()
sess.headers.update({"User-Agent":"kg-colab/1.2","Accept":"application/json","Accept-Encoding":"identity"})
sess.trust_env = False
TIMEOUT=30

def as_iso(d):
    if not d: return None
    try: return datetime.fromisoformat(str(d).replace("Z","+00:00")).date().isoformat()
    except: return str(d).split("T",1)[0]

def within_window(pubd, d0, d1):
    if not pubd: return False
    return (pubd >= d0) and (pubd <= d1)

# -------------------------
# Start Ollama (local) + pull model
# -------------------------
print("Setting up Ollama…")
os.environ["OLLAMA_HOST"] = "0.0.0.0:11434"
os.environ["OLLAMA_ORIGINS"] = "*"

def start_ollama():
    try:
        r = requests.get("http://127.0.0.1:11434", timeout=2)
        return True
    except Exception:
        pass
    subprocess.Popen(["bash","-lc","curl -fsSL https://ollama.com/install.sh -o install.sh && bash install.sh >/dev/null 2>&1 || true && nohup ollama serve >/dev/null 2>&1 &"],
                     stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    time.sleep(6)
    for _ in range(12):
        try:
            if requests.get("http://127.0.0.1:11434", timeout=2).ok:
                return True
        except Exception:
            time.sleep(1)
    return False

if start_ollama():
    print("Ollama API: OK")
    subprocess.run(["bash","-lc",f"ollama pull {OLLAMA_MODEL}"], check=False)
else:
    print("⚠️ Ollama not available; will run rule-based only.")

def ollama_available():
    try:
        return requests.get("http://127.0.0.1:11434/api/tags", timeout=3).ok
    except Exception:
        return False

# -------------------------
# PubMed (E-utilities JSON)
# -------------------------
def pubmed_search(term, dfrom, dto, retmax=200000):
    base = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    params = {
        "db":"pubmed", "term": term, "retmode":"json",
        "retmax": retmax, "datetype":"pdat", "mindate": dfrom, "maxdate": dto
    }
    r = sess.get(base, params=params, timeout=TIMEOUT)
    try:
        data = r.json()
    except Exception:
        # fallback parse: strip control chars
        txt = r.text.replace("\n"," ").replace("\r"," ")
        data = json.loads(txt)
    ids = data.get("esearchresult", {}).get("idlist", [])
    return ids

def pubmed_fetch(pmids):
    if not pmids: return []
    base = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi"
    B=200
    for i in range(0, len(pmids), B):
        chunk = pmids[i:i+B]
        r = sess.get(base, params={"db":"pubmed","id":",".join(chunk),"retmode":"json"}, timeout=TIMEOUT)
        try:
            data = r.json()
        except Exception:
            txt = r.text.replace("\n"," ").replace("\r"," ")
            data = json.loads(txt)
        uids = data.get("result",{}).get("uids",[])
        for uid in uids:
            it = data["result"].get(uid,{})
            title = it.get("title") or ""
            ab = it.get("elocationid","")
            # fetch abstract via efetch (optional; many esummary lack abstracts)
            abstr=""
            try:
                ef = sess.get("https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi",
                              params={"db":"pubmed","id":uid,"retmode":"xml"}, timeout=TIMEOUT)
                # quick/dirty abstract parse
                m=re.search(r"<AbstractText[^>]*>([\s\S]*?)</AbstractText>", ef.text)
                if m: abstr = re.sub("<.*?>","",m.group(1))
            except Exception: pass
            pubd = it.get("pubdate") or it.get("sortpubdate") or ""
            yield {
                "server":"pubmed","pmid":uid,"title":title,"abstract":abstr,
                "published_date":as_iso(pubd),"url":f"https://pubmed.ncbi.nlm.nih.gov/{uid}/"
            }

# -------------------------
# xRxiv (bioRxiv/medRxiv) JSON
# -------------------------
def biorxiv_fetch(server, start_date, end_date):
    base=f"https://api.biorxiv.org/details/{server}/{start_date}/{end_date}"
    cursor=0; seen=set()
    while True:
        r = sess.get(f"{base}/{cursor}", timeout=TIMEOUT)
        r.raise_for_status()
        data=r.json()
        coll=data.get("collection",[])
        if not coll: break
        for rec in coll:
            yield {
                "server":server,
                "id": rec.get("doi") or rec.get("biorxiv_doi") or rec.get("medrxiv_doi"),
                "title": rec.get("title",""),
                "abstract": rec.get("abstract",""),
                "published_date": as_iso(rec.get("date")),
                "url": rec.get("jatsxml") or rec.get("url")
            }
        nxt=data.get("next_cursor")
        if nxt is None or nxt in seen: break
        seen.add(nxt); cursor=nxt
        time.sleep(0.15)

# ChemRxiv via Figshare
def _iso_date_only(s):
    try: return datetime.fromisoformat(s.replace("Z","+00:00")).date().isoformat()
    except: return s.split("T",1)[0] if "T" in s else s

def chemrxiv_fetch(start_date, end_date):
    base1="https://api.figshare.com/v2/articles/search"
    seen=set(); page=1
    while True:
        params={"search_for":"ChemRxiv","page":page,"page_size":200,"order":"published_date","order_direction":"desc",
                "published_since":start_date,"published_until":end_date,"institution":"ChemRxiv"}
        r = sess.get(base1, params=params, timeout=TIMEOUT)
        if r.status_code in (400,404):
            break
        items=r.json()
        if not items: break
        wrote=0
        for it in items:
            aid=it.get("id")
            if aid in seen: continue
            seen.add(aid)
            pubd=_iso_date_only(it.get("published_date","") or (it.get("timeline") or {}).get("published",""))
            yield {
                "server":"chemrxiv","id":str(aid),
                "title": it.get("title",""),
                "abstract": it.get("description","") or "",
                "published_date": as_iso(pubd),
                "url": it.get("url_public_html") or it.get("url")
            }
            wrote+=1
        if wrote==0: break
        page+=1
        time.sleep(0.1)

# -------------------------
# IO helpers
# -------------------------
def gz_write(path, records):
    n=0
    with gzip.open(path, "wt", encoding="utf-8") as f:
        for r in records:
            f.write(json.dumps(r, ensure_ascii=False)+"\n")
            n+=1
    return n

def gz_iter(path):
    with gzip.open(path, "rt", encoding="utf-8", errors="ignore") as f:
        for line in f:
            try:
                yield json.loads(line)
            except Exception:
                continue

# -------------------------
# Crawl
# -------------------------
SEARCH_TERMS = [
    "(cancer OR carcinoma OR neoplasm OR oncology OR tumor)",
    "(mitochondria OR mitochondrial OR metabolism OR metabolic)",
]
print(f"Crawling PubMed for {DATE_FROM} → {DATE_TO}")
pm_all=[]
for term in SEARCH_TERMS:
    ids = pubmed_search(term, DATE_FROM, DATE_TO)
    print(f"PubMed ids for [{term}]:", len(ids))
    pm_all.extend(ids)
pm_all = list(dict.fromkeys(pm_all))
pubmed_records = list(pubmed_fetch(pm_all))
n = gz_write(os.path.join(RAW,"pubmed.jsonl.gz"), pubmed_records)
print("Wrote PubMed:", n)

if XRXIV_WINDOW_DAYS:
    dfrom = (datetime.fromisoformat(DATE_TO).date() - timedelta(days=XRXIV_WINDOW_DAYS)).isoformat()
    dto   = DATE_TO
    print("xRxiv window:", dfrom, "→", dto)
    bx = list(biorxiv_fetch("biorxiv", dfrom, dto))
    mx = list(biorxiv_fetch("medrxiv", dfrom, dto))
    cx = list(chemrxiv_fetch(dfrom, dto))
    gz_write(os.path.join(RAW,"biorxiv.jsonl.gz"), bx)
    gz_write(os.path.join(RAW,"medrxiv.jsonl.gz"), mx)
    gz_write(os.path.join(RAW,"chemrxiv.jsonl.gz"), cx)
    print("Wrote xRxiv:", len(bx), len(mx), len(cx))
else:
    print("Set XRXIV_WINDOW_DAYS to a positive number for fast windowed crawl.")
    bx=mx=cx=[]

# -------------------------
# Filter + dedupe
# -------------------------
def passes(rec):
    title=(rec.get("title") or "").lower()
    ab=(rec.get("abstract") or rec.get("description") or "").lower()
    return any(k in title or k in ab for k in KEYWORDS)

def normalize_rec(rec):
    out=dict(rec)
    out["published_date"]=as_iso(out.get("published_date") or out.get("date"))
    return out

all_paths=[os.path.join(RAW,"pubmed.jsonl.gz"), os.path.join(RAW,"biorxiv.jsonl.gz"),
           os.path.join(RAW,"medrxiv.jsonl.gz"), os.path.join(RAW,"chemrxiv.jsonl.gz")]
seen_keys=set(); filtered=[]
for p in all_paths:
    if not os.path.exists(p): continue
    for rec in gz_iter(p):
        r=normalize_rec(rec)
        if not within_window(r.get("published_date") or "1900-01-01", DATE_FROM, DATE_TO):
            continue
        if not passes(r): continue
        k = ((r.get("doi") or "").lower().strip(),
             (r.get("pmid") or r.get("id") or r.get("url") or "").lower().strip(),
             r.get("published_date") or "")
        if k in seen_keys: continue
        seen_keys.add(k)
        filtered.append(r)

FIL_COMBINED = os.path.join(FIL,"filtered_all.jsonl.gz")
gz_write(FIL_COMBINED, filtered)
print(f"Filtered kept: {len(filtered)} → {FIL_COMBINED}")

# -------------------------
# Triples: disease spans (gazetteer+regex) + priority list + optional LLM
# -------------------------
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span

DISEASE_TERMS = {
    "cancer","malignancy","neoplasm","tumor","tumour","carcinoma","sarcoma","lymphoma",
    "leukemia","leukaemia","melanoma","glioma","blastoma","myeloma","metastasis","metastases",
    "breast cancer","lung cancer","colorectal cancer","colon cancer","rectal cancer","prostate cancer",
    "pancreatic cancer","ovarian cancer","cervical cancer","endometrial cancer",
    "gastric cancer","stomach cancer","liver cancer","hepatocellular carcinoma","renal cancer",
    "kidney cancer","bladder cancer","thyroid cancer","head and neck cancer",
    "brain tumor","glioblastoma","meningioma","astrocytoma","medulloblastoma",
    "acute myeloid leukemia","acute lymphoblastic leukemia","chronic myeloid leukemia",
    "chronic lymphocytic leukemia","hodgkin lymphoma","non-hodgkin lymphoma","multiple myeloma",
    "basal cell carcinoma","squamous cell carcinoma","cutaneous melanoma","neuroblastoma",
    "ewing sarcoma","osteosarcoma","rhabdomyosarcoma","wilms tumor",
}
DISEASE_REGEX = re.compile(
    r"\b(?:[A-Za-z\-]+)(?:carcinoma|sarcoma|lymphoma|melanoma|glioma|blastoma|myeloma|leukemia|tumou?r|neoplasm|metastas(?:is|es))s?\b",
    flags=re.I
)
pm = PhraseMatcher(nlp.vocab, attr="LOWER")
pm.add("DISEASE_GAZ", [nlp.make_doc(t) for t in sorted(DISEASE_TERMS)])
pm.add("PRIORITY", [nlp.make_doc(t) for t in sorted(PRIORITY_ENTITIES)])

def disease_spans(span_or_doc):
    if isinstance(span_or_doc, Span):
        sent = span_or_doc; doc = sent.doc
        s_tok, e_tok = sent.start, sent.end; s_char = sent.start_char
    else:
        sent=None; doc=span_or_doc; s_tok, e_tok = 0, len(doc); s_char=0
    spans=[]
    for _, start, end in pm(doc):
        if start>=s_tok and end<=e_tok:
            spans.append(Span(doc, start, end, label="ENT"))
    taken={(sp.start,sp.end) for sp in spans}
    text = (sent.text if sent is not None else doc.text)
    for m in DISEASE_REGEX.finditer(text):
        cs = (s_char + m.start()) if sent is not None else m.start()
        ce = (s_char + m.end()) if sent is not None else m.end()
        sp = doc.char_span(cs, ce, alignment_mode="expand")
        if sp and sp.start>=s_tok and sp.end<=e_tok and (sp.start,sp.end) not in taken:
            spans.append(Span(doc, sp.start, sp.end, label="ENT"))
    spans.sort(key=lambda s: (s.start, -len(s)))
    merged=[]
    for s in spans:
        if not merged or s.start >= merged[-1].end:
            merged.append(s)
        else:
            prev=merged[-1]
            if (s.end-s.start) > (prev.end-prev.start):
                merged[-1]=s
    return merged

def alias_node(name: str) -> str:
    n = re.sub(r"\s+"," ", (name or "").strip()).lower()
    return ALIASES.get(n, n)

THINK_RE = re.compile(r"<think>.*?</think>", re.S|re.I)

def llm_triples(sent_text, concepts):
    if not ollama_available(): return []
    concepts = list(dict.fromkeys([c.strip() for c in concepts if c.strip()]))[:12]
    if len(concepts)<2: return []
    prompt = (
        "Extract a JSON array of triples with keys subject, relation, object. "
        "Subject and object MUST be chosen from this exact set (string match):\n"
        f"{json.dumps(concepts, ensure_ascii=False)}\n\n"
        f"Sentence: {json.dumps(sent_text)}\n"
        "Return only JSON."
    )
    try:
        r = requests.post("http://127.0.0.1:11434/completions",
                          json={"model": OLLAMA_MODEL, "prompt": prompt, "max_tokens": 256, "temperature": 0.0},
                          timeout=60)
        r.raise_for_status()
        txt = r.json().get("choices",[{}])[0].get("text","")
        txt = THINK_RE.sub("", txt).strip()
        m = re.search(r"\[[\s\S]*\]", txt)
        if not m: return []
        data = json.loads(m.group(0))
        out=[]
        for d in data:
            if not isinstance(d, dict): continue
            s=d.get("subject",""); rrel=d.get("relation",""); o=d.get("object","")
            if s and rrel and o:
                out.append({"subject":s,"relation":rrel,"object":o,"method":"llm"})
        return out
    except Exception:
        return []

def rule_triples(sent, ents):
    out=[]
    # lightweight: use sentence root lemma if it's a verb-like
    root = sent.root
    rel = root.lemma_ if root.pos_ in ("VERB","AUX") else "related_to"
    for i in range(len(ents)):
        for j in range(i+1, len(ents)):
            a=ents[i].text; b=ents[j].text
            out.append({"subject":a,"relation":rel,"object":b,"method":"rule"})
            out.append({"subject":b,"relation":rel,"object":a,"method":"rule"})
    return out

def get_record_text(rec):
    title = rec.get("title") or ""
    abstract = rec.get("abstract") or rec.get("description") or ""
    if not USE_FULLTEXT:
        return (title + "\n\n" + abstract).strip()
    # best-effort PDF fetch if url looks like a PDF
    u = rec.get("url") or ""
    txt = (title + "\n\n" + abstract).strip()
    if ".pdf" in u.lower():
        try:
            rb = sess.get(u, timeout=20).content
            pdf_txt = pdf_extract_text(io.BytesIO(rb), maxpages=40) or ""
            if len(pdf_txt) > len(txt)*2:
                return pdf_txt
        except Exception:
            pass
    return txt

# Build graph + write triples audit
TRIPLES_GZ = os.path.join(OUT,"triples.jsonl.gz")
G = nx.DiGraph()
first_labels = {}
t_out = gzip.open(TRIPLES_GZ, "wt", encoding="utf-8")
paper_count=0

print("Extracting triples…")
for rec in tqdm(gz_iter(FIL_COMBINED), total=len(filtered) if filtered else None):
    paper_count += 1
    if MAX_PAPERS_TRIPLES and paper_count > MAX_PAPERS_TRIPLES:
        break
    text = get_record_text(rec)
    if not text: continue
    doc = nlp(text)
    pid = str(rec.get("pmid") or rec.get("doi") or rec.get("id") or rec.get("url") or paper_count)
    for sent in doc.sents:
        ents = disease_spans(sent)
        # make sure priority strings present in sentence are included as entities
        s_lower = sent.text.lower()
        for pstr in PRIORITY_ENTITIES:
            if pstr.lower() in s_lower and pstr not in [e.text.lower() for e in ents]:
                # add virtual span via string pos
                m = re.search(re.escape(pstr), sent.text, re.I)
                if m:
                    sp = doc.char_span(sent.start_char+m.start(), sent.start_char+m.end(), alignment_mode="expand")
                    if sp:
                        ents.append(sp)
        if len(ents) < 2:
            continue
        r1 = rule_triples(sent, ents)
        r2 = llm_triples(sent.text, [e.text for e in ents])
        for t in itertools.chain(r1, r2):
            s_raw, o_raw, rel_raw = t["subject"], t["object"], t["relation"]
            if not s_raw or not o_raw or not rel_raw:
                continue
            s = alias_node(s_raw); o = alias_node(o_raw); rel = re.sub(r"\s+"," ", rel_raw.strip()).lower()
            if s == o: continue
            first_labels.setdefault(("node", s), s_raw)
            first_labels.setdefault(("node", o), o_raw)
            first_labels.setdefault(("rel",  rel), rel_raw)

            # add
            G.add_node(s, label=first_labels[("node", s)])
            G.add_node(o, label=first_labels[("node", o)])
            if G.has_edge(s,o):
                G[s][o]["weight"] = int(G[s][o].get("weight",0)) + 1
                G[s][o].setdefault("relations", Counter())
                G[s][o]["relations"][rel] += 1
            else:
                G.add_edge(s,o, weight=1, relations=Counter({rel:1}))
            # audit
            t_out.write(json.dumps({
                "paper_id": pid, "sentence": sent.text,
                "subject": s_raw, "relation": rel_raw, "object": o_raw,
                "method": t.get("method","")
            }, ensure_ascii=False) + "\n")
t_out.close()
print(f"Triples → {TRIPLES_GZ}")
print(f"Graph so far: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

# -------------------------
# Normalize counts → probabilities p
# -------------------------
import numpy as np
def normalize_edge_weights(G, alpha=NORM_ALPHA, in_attr="weight", out_attr="p"):
    for _,_,d in G.edges(data=True):
        c = float(d.get(in_attr, 1.0))
        p = 1.0 - np.exp(-c / float(alpha))
        d[out_attr] = float(p)
        # best relation label
        rels = d.get("relations", {})
        if rels:
            best_rel = max(rels.items(), key=lambda kv: kv[1])[0]
            d["relation_best"] = first_labels.get(("rel",best_rel), best_rel)
normalize_edge_weights(G)

# prune low-count edges
to_drop=[(u,v) for u,v,d in G.edges(data=True) if int(d.get("weight",1)) < MIN_EDGE_COUNT]
G.remove_edges_from(to_drop)

# keep top-N nodes by weighted degree
if G.number_of_nodes() > TOP_N_NODES:
    degs = dict(G.degree(weight="weight"))
    keep = set(sorted(degs, key=degs.get, reverse=True)[:TOP_N_NODES])
    G = G.subgraph(keep).copy()

# analysis graph without super-generic nodes
def pruned_for_analysis(G):
    H = G.copy()
    by_label = {n: G.nodes[n].get("label","").lower() for n in G.nodes}
    drop = [n for n,lbl in by_label.items() if lbl in STOP_NODES]
    H.remove_nodes_from(drop)
    return H
G_an = pruned_for_analysis(G)

# -------------------------
# Category-theory-ish analyses
# -------------------------
def shared_precursors(G, outcome1, outcome2, max_depth=2):
    if outcome1 not in G or outcome2 not in G:
        return set()
    preds1 = {p for p in nx.ancestors(G, outcome1) if nx.shortest_path_length(G, p, outcome1) <= max_depth}
    preds2 = {p for p in nx.ancestors(G, outcome2) if nx.shortest_path_length(G, p, outcome2) <= max_depth}
    return preds1 & preds2

def max_product_paths(G, cutoff=4, pkey="p"):
    strength={}
    for src in G.nodes:
        for tgt in G.nodes:
            if src==tgt or not nx.has_path(G, src, tgt):
                continue
            best=0.0
            for path in nx.all_simple_paths(G, src, tgt, cutoff=cutoff):
                w=1.0
                for u,v in zip(path, path[1:]):
                    w *= float(G[u][v].get(pkey, 0.5))
                if w>best: best=w
            if best>0:
                strength[(src,tgt)] = best
    return strength

ACT_PAT = re.compile(r"(activat|induc|upregulat|promot|increase|enhanc|stimulat)", re.I)
INH_PAT = re.compile(r"(inhibit|block|suppress|downregulat|decreas|attenuat)", re.I)
def rel_polarity(r: str) -> int:
    if not r: return 0
    if ACT_PAT.search(r): return +1
    if INH_PAT.search(r): return -1
    return 0

def find_contradictions(G):
    contras=[]
    for a in G.nodes:
        for b in G.successors(a):
            for c in G.successors(a):
                if b==c: continue
                Db=set(G.successors(b)); Dc=set(G.successors(c))
                for d in Db & Dc:
                    p1 = rel_polarity(G[a][b].get("relation_best") or "")
                    p2 = rel_polarity(G[b][d].get("relation_best") or "")
                    q1 = rel_polarity(G[a][c].get("relation_best") or "")
                    q2 = rel_polarity(G[c][d].get("relation_best") or "")
                    net1 = p1*p2; net2 = q1*q2
                    if net1 != 0 and net2 != 0 and np.sign(net1) != np.sign(net2):
                        contras.append((a,b,c,d))
    return contras

# Node helpers
def label_for(n): return G.nodes[n].get("label", n)

# Run a few canonical queries
o1, o2 = alias_node("p53 pathway"), alias_node("apoptosis")
shared_1 = shared_precursors(G_an, o1, o2, max_depth=2)

o3, o4 = alias_node("mitochondrial dysfunction"), alias_node("apoptosis")
shared_2 = shared_precursors(G_an, o3, o4, max_depth=2)

print(f"Shared precursors for [{label_for(o1)}] & [{label_for(o2)}]: {len(shared_1)}")
print(f"Shared precursors for [{label_for(o3)}] & [{label_for(o4)}]: {len(shared_2)}")

strengths = max_product_paths(G_an, cutoff=4)
top_strengths = sorted(strengths.items(), key=lambda kv: -kv[1])[:10]
print("Top max-product links (up to 10):")
for (s,t),val in top_strengths:
    print(f"  {label_for(s)} → {label_for(t)} : {val:.4f}")

contras = find_contradictions(G_an)
print("Potential contradictions:", len(contras))

# -------------------------
# Exports (CSV / GraphML / GEXF / HTML)
# -------------------------
EDGES_CSV = os.path.join(OUT,"kg_edges.csv")
GRAPHML   = os.path.join(OUT,"kg.graphml")
GEXF      = os.path.join(OUT,"kg.gexf")
HTML_OUT  = os.path.join(OUT,"kg.html")

# flatten for GraphML
def make_graphml_safe(G):
    H=G.copy()
    for _,_,d in H.edges(data=True):
        if "relations" in d:
            d["relations_str"] = "; ".join(
                f"{first_labels.get(('rel',k),k)}:{c}" for k,c in d["relations"].items()
            )
            del d["relations"]
        d["weight"] = int(d.get("weight",1))
        d["p"] = float(d.get("p",0.5))
        d["relation_best"] = d.get("relation_best","")
    for n, data in H.nodes(data=True):
        if "label" in data and not isinstance(data["label"], (str,int,float,bool)):
            data["label"] = str(data["label"])
    return H

edge_rows=[]
for u,v,d in G.edges(data=True):
    rels = "; ".join(f"{first_labels.get(('rel',k),k)}×{c}" for k,c in d.get("relations",{}).items())
    edge_rows.append({
        "subject": G.nodes[u].get("label",u),
        "object":  G.nodes[v].get("label",v),
        "count":   int(d.get("weight",1)),
        "p":       float(d.get("p",0.5)),
        "relation_best": d.get("relation_best",""),
        "relations": rels
    })
pd.DataFrame(edge_rows).to_csv(EDGES_CSV, index=False)

G_safe = make_graphml_safe(G)
nx.write_graphml(G_safe, GRAPHML)
nx.write_gexf(G_safe, GEXF)

# -------------------------
# Shared-precursor subgraph viz (mito dysf & apoptosis)
# -------------------------
def subgraph_from_nodes(G, nodes, add_targets=None):
    keep=set(nodes)
    if add_targets: keep |= set(add_targets)
    return G.subgraph(keep).copy()

try:
    H = subgraph_from_nodes(G_an, shared_2, add_targets=[o3,o4])
    net = Network(height="700px", width="100%", directed=True, notebook=True, cdn_resources="in_line")
    net.barnes_hut(gravity=-20000, central_gravity=0.15, spring_length=160, spring_strength=0.01, damping=0.85)
    deg = dict(H.degree(weight="weight"))
    def sz(x): return max(8, 10 * (1+math.log1p(x)))
    for n,d in H.nodes(data=True):
        label = d.get("label",n)
        color = "#4F8DD3" if n==o3 else ("#D34F6A" if n==o4 else "#9AA0A6")
        net.add_node(n, label=label, value=sz(deg.get(n,1)), color=color)
    for u,v,d in H.edges(data=True):
        title = d.get("relation_best","") or d.get("relations_str","")
        net.add_edge(u,v, value=float(d.get("p",0.5)), title=title)
    html = net.generate_html()
    with open(HTML_OUT,"w",encoding="utf-8") as f: f.write(html)
    display(HTML(html))
    print("Saved HTML:", HTML_OUT)
except Exception as e:
    print("Shared-precursor viz failed:", e)

print("Saved:", TRIPLES_GZ, EDGES_CSV, GRAPHML, GEXF, HTML_OUT)


Installing deps…
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 701.5 kB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 2.0.8 requires opencv-python-headless>=4.9.0.80, which is not installed.
dopamine-rl 4.1.2 requires opencv-python>=3.4.8.29, which is not installed.
albucore 0.0.24 requires opencv-python-headless>=4.9.0.80, which is not installed.
google-colab 1.0.0 requires ipython==7.34.0, but you have ipython 9.4.0 which is incompatible.
google-colab 1.0.0 requires requests==2.32.3, but you have requests 2.32.4 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machi

 65%|██████▌   | 5000/7667 [02:16<01:12, 36.57it/s]


Triples → /content/kg_out/out/triples.jsonl.gz
Graph so far: 209 nodes, 2068 edges
Shared precursors for [p53 pathway] & [Apoptosis]: 39
Shared precursors for [mitochondrial dysfunction] & [Apoptosis]: 96
Top max-product links (up to 10):
  prostate cancer → prostate : 1.0000
  prostate → prostate cancer : 1.0000
  proliferation → Apoptosis : 0.9983
  Apoptosis → proliferation : 0.9983
  Apoptosis → ROS : 0.9727
  ROS → Apoptosis : 0.9727
  proliferation → ROS : 0.9711
  ROS → proliferation : 0.9711
  Apoptosis → Mitochondrial : 0.9631
  Mitochondrial → Apoptosis : 0.9631
Potential contradictions: 72


Saved HTML: /content/kg_out/out/kg.html
Saved: /content/kg_out/out/triples.jsonl.gz /content/kg_out/out/kg_edges.csv /content/kg_out/out/kg.graphml /content/kg_out/out/kg.gexf /content/kg_out/out/kg.html


In [ ]:
# -------------------
# Post-process KG with evidence filter + rich output
# -------------------

MIN_WEIGHT = 3  # tune: minimum evidence count to keep an edge

# Prune weak edges
edges_to_remove = [(u, v) for u, v, d in G.edges(data=True) if d.get("weight", 0) < MIN_WEIGHT]
G.remove_edges_from(edges_to_remove)
print(f"Removed {len(edges_to_remove)} edges with weight < {MIN_WEIGHT}")
print(f"Graph after filtering: {len(G.nodes())} nodes, {len(G.edges())} edges\n")

# Helper: get examples for an edge
def get_edge_examples(u, v, max_n=3):
    sents = G[u][v].get("examples", [])
    return sents[:max_n]

# -------------------
# Shared precursors examples
# -------------------
def shared_precursors(a, b, max_n=5):
    a_preds = set(G.predecessors(a))
    b_preds = set(G.predecessors(b))
    shared = list(a_preds & b_preds)
    shared.sort(key=lambda n: len(G[n][a].get("examples", [])) + len(G[n][b].get("examples", [])), reverse=True)
    return shared[:max_n]

for query_a, query_b in [("p53 pathway", "Apoptosis"), ("mitochondrial dysfunction", "Apoptosis")]:
    shared = shared_precursors(query_a, query_b)
    print(f"Shared precursors for [{query_a}] & [{query_b}] (≥{MIN_WEIGHT} evidences): {len(shared)}")
    for node in shared:
        print(f"  {node}")
        print(f"    → {query_a}: {get_edge_examples(node, query_a, 2)}")
        print(f"    → {query_b}: {get_edge_examples(node, query_b, 2)}")
    print()

# -------------------
# Top max-product links with examples
# -------------------
def max_product_links(G, top_n=10):
    scores = []
    for u, v, d in G.edges(data=True):
        w_uv = d.get("weight", 0)
        w_vu = G[v][u]["weight"] if G.has_edge(v, u) else 0
        score = w_uv * w_vu
        if score > 0:
            scores.append((score, u, v))
    scores.sort(reverse=True)
    return scores[:top_n]

print(f"Top max-product links (≥{MIN_WEIGHT} evidences):")
for score, u, v in max_product_links(G):
    print(f"  {u} → {v} : {score:.4f}")
    exs = get_edge_examples(u, v, 2)
    if exs:
        print("    Examples:", exs)
print()

# -------------------
# Potential contradictions (with example sentences)
# -------------------
contradictions = []
for u, v, d in G.edges(data=True):
    if "NEG" in d.get("polarity", ""):  # adjust depending on your schema
        contradictions.append((u, v, d.get("examples", [])[:2]))

print(f"Potential contradictions: {len(contradictions)}")
for u, v, exs in contradictions[:5]:
    print(f"  {u} ↔ {v}")
    for ex in exs:
        print(f"    Example: {ex}")


In [ ]:
# =========================
# Evidence-driven postprocessing
# =========================
import gzip, json, math, itertools
from collections import defaultdict, Counter
import networkx as nx

# --- tweak these if your filenames or outcomes differ ---
TRIPLES_GZ = "/content/kg_out/out/triples.jsonl.gz"  # your triples audit file
OUTCOME_1  = "p53 pathway"
OUTCOME_2  = "Apoptosis"
OUTCOME_3  = "mitochondrial dysfunction"
MAX_SENTENCES_PER_EDGE = 2
TOP_K_SHARED = 5
TOP_K_CONTRADICTIONS = 5
TOP_K_EXPLAIN = 10     # explain this many top max-product links
PATH_CUTOFF = 4

def canon(x: str) -> str:
    return " ".join((x or "").split()).lower()

# --- load evidence from triples.gz ---
evidence_by_edge = defaultdict(list)   # (s,o) -> [{sent, paper, method, relation}]
evidence_by_trip = defaultdict(list)   # (s,r,o) -> [ ... ]
unique_edges_counter = Counter()

try:
    with gzip.open(TRIPLES_GZ, "rt", encoding="utf-8") as f:
        for line in f:
            try:
                rec = json.loads(line)
            except json.JSONDecodeError:
                continue
            s_raw = rec.get("subject") or rec.get("subj") or ""
            o_raw = rec.get("object")  or rec.get("obj")  or ""
            r_raw = rec.get("relation") or rec.get("predicate") or ""
            s, o, r = canon(s_raw), canon(o_raw), canon(r_raw)
            ev = {
                "sent": rec.get("sentence",""),
                "paper": rec.get("paper_id") or rec.get("pmid") or rec.get("doi") or "?",
                "method": rec.get("method",""),
                "relation": r_raw or ""
            }
            evidence_by_edge[(s,o)].append(ev)
            evidence_by_trip[(s,r,o)].append(ev)
            unique_edges_counter[(s,o)] += 1
except FileNotFoundError:
    print(f"⚠️ Evidence file not found: {TRIPLES_GZ} — examples will be sparse.")

# handy: get original label from your graph (we stored 'label' on nodes)
def label_of(node_id):
    return G.nodes[node_id].get("label", node_id)

def sample_evidence(s, o, rel=None, k=MAX_SENTENCES_PER_EDGE):
    s_c, o_c = canon(s), canon(o)
    if rel is None:
        pool = evidence_by_edge.get((s_c,o_c), [])
    else:
        pool = evidence_by_trip.get((s_c, canon(rel), o_c), [])
        if not pool:  # fall back to any relation
            pool = evidence_by_edge.get((s_c,o_c), [])
    out = []
    for ev in pool[:k]:
        sent = ev.get("sent","").strip()
        paper = ev.get("paper","?")
        meth  = ev.get("method","")
        out.append(f'  • “…{sent}”  [paper: {paper}; method: {meth}]')
    return out

# ---- 1) Shared precursors WITH EXAMPLES ----
def shared_precursors_examples(G, outcome1, outcome2, max_depth=2, top_k=TOP_K_SHARED):
    a, b = canon(outcome1), canon(outcome2)
    if a not in G or b not in G:
        print(f"⚠️ Outcome not in graph: {outcome1 if a not in G else outcome2}")
        return []
    # bounded ancestors by shortest path length <= max_depth
    preds1 = {p for p in nx.ancestors(G, a) if nx.shortest_path_length(G, p, a) <= max_depth}
    preds2 = {p for p in nx.ancestors(G, b) if nx.shortest_path_length(G, p, b) <= max_depth}
    shared = list(preds1 & preds2)
    # rank: sum of outgoing weights toward both outcomes (via nearest neighbor if multi-step)
    def rank_score(p):
        s=0.0
        # best edge on first hop from p toward a/b (if multi-step, we still sample first hop evidence)
        for tgt in [a,b]:
            try:
                path = nx.shortest_path(G, p, tgt, weight=None)
                if len(path)>=2:
                    u,v = path[0], path[1]
                    s += G[u][v].get("weight",1.0)
            except nx.NetworkXNoPath:
                pass
        return s
    shared.sort(key=rank_score, reverse=True)
    results=[]
    for p in shared[:top_k]:
        row = {"precursor": label_of(p), "examples_to": {}}
        for tgt_name, tgt in [(outcome1,a), (outcome2,b)]:
            try:
                path = nx.shortest_path(G, p, tgt)
                hop = (path[0], path[1]) if len(path)>=2 else None
            except nx.NetworkXNoPath:
                hop=None
            ev_lines = []
            if hop:
                u,v = hop
                # try per-edge relation list from original graph (if present)
                rels = []
                if "relations" in G[u][v]:
                    rels = list(G[u][v]["relations"].keys())
                # sample evidence (try the main relation first if available)
                if rels:
                    ev_lines = sample_evidence(label_of(u), label_of(v), rel=rels[0])
                if not ev_lines:
                    ev_lines = sample_evidence(label_of(u), label_of(v))
            row["examples_to"][tgt_name] = {
                "first_hop": f"{label_of(p)} → {label_of(path[1])}" if hop else "(no hop)",
                "sentences": ev_lines[:MAX_SENTENCES_PER_EDGE]
            }
        results.append(row)
    return results

print("\n=== Shared precursors with examples ===")
for (o1,o2) in [(OUTCOME_1, OUTCOME_2), (OUTCOME_3, OUTCOME_2)]:
    ex = shared_precursors_examples(G, o1, o2, max_depth=2, top_k=TOP_K_SHARED)
    print(f"\nTop {min(TOP_K_SHARED,len(ex))} shared precursors for [{o1}] & [{o2}]")
    for i,row in enumerate(ex,1):
        print(f"{i}. {row['precursor']}")
        for tgt,info in row["examples_to"].items():
            print(f"   ↳ toward {tgt}: {info['first_hop']}")
            for s in info["sentences"]:
                print("     " + s)

# ---- 2) Contradictions WITH SENTENCES (same (s,o) but multiple relation labels) ----
def contradictory_pairs(G, top_k=TOP_K_CONTRADICTIONS):
    items=[]
    for u,v,data in G.edges(data=True):
        rels=[]
        if "relations" in data and data["relations"]:
            rels = list(data["relations"].keys())
        elif "relations_str" in data:
            rels = [r.split(":")[0] for r in str(data["relations_str"]).split(";") if r.strip()]
        if len(set(rels)) >= 2:
            items.append((u,v,rels,int(data.get("weight",1))))
    items.sort(key=lambda x: -x[3])
    return items[:top_k]

print("\n=== Potential contradictions (with example sentences) ===")
contr = contradictory_pairs(G, TOP_K_CONTRADICTIONS)
if not contr:
    print("None found at this threshold.")
else:
    for i,(u,v,rels,w) in enumerate(contr,1):
        print(f"{i}. {label_of(u)} → {label_of(v)}  (weight={w})")
        # show up to 2 relations w/ sentences
        for rel in list(dict.fromkeys(rels))[:2]:
            print(f"   • relation: {rel}")
            lines = sample_evidence(label_of(u), label_of(v), rel=rel, k=MAX_SENTENCES_PER_EDGE)
            if not lines:
                lines = sample_evidence(label_of(u), label_of(v), rel=None, k=MAX_SENTENCES_PER_EDGE)
            for s in lines:
                print("     " + s)

# ---- 3) Explain TOP max-product links by showing best path + per-hop evidence ----
def best_path_and_score(G, s, t, cutoff=PATH_CUTOFF):
    best=0.0; best_path=None
    for path in nx.all_simple_paths(G, s, t, cutoff=cutoff):
        prod=1.0
        for a,b in zip(path, path[1:]):
            prod *= float(G[a][b].get("weight",1.0))
        if prod > best:
            best, best_path = prod, path
    return best, best_path

def max_product_pairs(G, cutoff=PATH_CUTOFF, top_k=TOP_K_EXPLAIN):
    scores=[]
    nodes=list(G.nodes)
    for i,s in enumerate(nodes):
        for t in nodes:
            if t==s: continue
            if not nx.has_path(G, s, t): continue
            sc, path = best_path_and_score(G, s, t, cutoff=cutoff)
            if sc>0 and path:
                scores.append(((s,t), sc, path))
    scores.sort(key=lambda x: -x[1])
    return scores[:top_k]

print("\n=== Explanations for top max-product links ===")
top_links = max_product_pairs(G, cutoff=PATH_CUTOFF, top_k=TOP_K_EXPLAIN)
for i,((s,t),score,path) in enumerate(top_links,1):
    human_path = " → ".join(label_of(n) for n in path)
    print(f"{i}. {label_of(s)} → {label_of(t)}  | score={score:.4g}  | path: {human_path}")
    # why high? Many supporting sentences (weights) and short path
    hop_infos=[]
    for a,b in zip(path, path[1:]):
        w = int(G[a][b].get("weight",1))
        # pick a representative relation name if available
        rel = None
        if "relations" in G[a][b] and G[a][b]["relations"]:
            rel = next(iter(G[a][b]["relations"].keys()))
        hop_label = f"{label_of(a)} → {label_of(b)}"
        print(f"   - hop {hop_label}  (w={w}{'; rel='+rel if rel else ''})")
        lines = sample_evidence(label_of(a), label_of(b), rel=rel, k=MAX_SENTENCES_PER_EDGE)
        for snt in lines:
            print("       " + snt)
    # brief rationale
    L = len(path)-1
    print(f"   Rationale: score is product of hop weights across {L} hop(s); high edge counts and short paths boost this.\n")



=== Shared precursors with examples ===

Top 5 shared precursors for [p53 pathway] & [Apoptosis]
1. proliferation
   ↳ toward p53 pathway: proliferation → ROS
       • “…Further, the effect of KPNA2 on the LUAD cells in vitro was determined by the EdU assay for proliferation and flow cytometric (FCM) analysis to assess apoptosis and ROS levels.”  [paper: 40785975; method: rule]
   ↳ toward Apoptosis: proliferation → Apoptosis
       • “…FQs such as ciprofloxacin, moxifloxacin, and levofloxacin are found to enhance the anti-tumor effect of a wide range of FDA-approved chemotherapeutics through multiple mechanisms including cell cycle arrest, apoptosis induction, anti-proliferation, and modulation of EMT.”  [paper: 40788483; method: rule]
       • “…Moreover, SMARCA4 degradation significantly reduced proliferation and induced apoptosis of SOX11-positive MCL cells, highlighting AU-15330 as a promising therapeutic approach for patients who may relapse from current target therapies in MCL.

In [ ]:
# ============================
# Interactive PyVis KG (with rich tooltips)
# ============================
from pyvis.network import Network
from IPython.display import HTML, display
import html, math

HTML_OUT = "/content/kg_out/interactive_kg.html"
MIN_EDGE_WEIGHT_FOR_DISPLAY = 2   # hide very weak edges
EXAMPLES_PER_EDGE = 2             # how many example sentences to show per edge
NODE_SIZE_SCALE = 10

def _short(s, n=240):
    s = s.strip()
    return (s[:n] + "…") if len(s) > n else s

def _edge_examples(u, v, k=EXAMPLES_PER_EDGE):
    exs = G[u][v].get("examples", []) or []
    return [html.escape(_short(x)) for x in exs[:k]]

def _node_color(n):
    lab = (G.nodes[n].get("label") or str(n)).lower()
    # quick, readable palette by coarse type guess
    if any(w in lab for w in ["cancer","carcinoma","tumor","tumour","neoplasm","sarcoma","lymphoma","leukemia","melanoma"]):
        return "#ef5350"  # disease
    if any(w in lab for w in ["apoptosis","proliferation","metastasis","autophagy","inflammation","dna damage","senescence"]):
        return "#42a5f5"  # process
    if any(w in lab for w in ["mitochond", "metab", "glycolysis", "ros", "respirat", "krebs", "tca"]):
        return "#66bb6a"  # mito/metabolism
    return "#9aa0a6"      # other

# Degree-based sizing
deg = dict(G.degree(weight="weight"))
in_deg = dict(G.in_degree(weight="weight")) if isinstance(G, type(nx.DiGraph())) else deg
out_deg = dict(G.out_degree(weight="weight")) if isinstance(G, type(nx.DiGraph())) else deg
def _size_fn(x): return max(8, NODE_SIZE_SCALE * (1 + math.log1p(x)))

# Build a viz subgraph filtered by weight
H = G.copy()
H.remove_edges_from([(u, v) for u, v, d in H.edges(data=True) if d.get("weight", 0) < MIN_EDGE_WEIGHT_FOR_DISPLAY])
H.remove_nodes_from([n for n in list(H.nodes()) if H.degree(n) == 0])

net = Network(height="780px", width="100%", directed=H.is_directed(), notebook=True, cdn_resources="in_line")
# physics + interaction (valid JSON string)
net.set_options("""
{
  "interaction": { "hover": true, "tooltipDelay": 120, "zoomView": true },
  "physics": {
    "stabilization": { "enabled": true, "iterations": 500 },
    "barnesHut": { "gravitationalConstant": -18000, "centralGravity": 0.2, "springLength": 150, "springConstant": 0.02, "damping": 0.8 }
  },
  "edges": { "smooth": { "type": "dynamic" }, "arrows": { "to": { "enabled": true } } },
  "layout": { "improvedLayout": true }
}
""")

# Add nodes with rich tooltips
for n, data in H.nodes(data=True):
    label = data.get("label", n)
    w = deg.get(n, 1.0)
    size = _size_fn(w)
    indeg = in_deg.get(n, 0.0); outdeg = out_deg.get(n, 0.0)
    color = _node_color(n)
    title = f"""
    <div style='max-width:420px'>
      <b>{html.escape(str(label))}</b><br/>
      <small>Weighted degree: {w}; in: {indeg}; out: {outdeg}</small><br/>
      <hr style='border:none;border-top:1px solid #ddd;margin:6px 0'/>
      <i>Tip:</i> click a connected edge to see example sentences.
    </div>
    """
    net.add_node(n, label=str(label), title=title, color=color, value=size)

# Add edges with example sentences in tooltip
for u, v, d in H.edges(data=True):
    w = int(d.get("weight", 1))
    rels = d.get("relations_str", "") or ""
    exs = _edge_examples(u, v, EXAMPLES_PER_EDGE)
    ex_html = ""
    if exs:
        ex_html = "<br/>".join([f"<div style='margin:4px 0'>“{e}”</div>" for e in exs])
    title = f"""
    <div style='max-width:520px'>
      <b>{html.escape(str(H.nodes[u].get('label', u)))} → {html.escape(str(H.nodes[v].get('label', v)))}</b><br/>
      <small>weight={w}{('; ' + html.escape(rels)) if rels else ''}</small>
      {("<hr style='border:none;border-top:1px solid #ddd;margin:6px 0'/>" + ex_html) if exs else ""}
    </div>
    """
    net.add_edge(u, v, value=w, title=title)

# Generate inline HTML and attach a legend + search box
html_doc = net.generate_html()

legend = """
<div id="kg-legend" style="position:fixed; right:12px; bottom:12px; z-index:9999; background:#fff; border:1px solid #ddd; border-radius:10px; box-shadow:0 2px 8px rgba(0,0,0,0.08); padding:10px 12px; font: 12px/1.2 system-ui, -apple-system, Segoe UI, Roboto, sans-serif;">
  <div style="margin-bottom:8px; font-weight:600;">Legend</div>
  <div style="display:flex; gap:10px; align-items:center; margin:4px 0;">
    <span style="width:12px;height:12px;background:#ef5350;border-radius:50%;display:inline-block;"></span> Disease
  </div>
  <div style="display:flex; gap:10px; align-items:center; margin:4px 0;">
    <span style="width:12px;height:12px;background:#42a5f5;border-radius:50%;display:inline-block;"></span> Process
  </div>
  <div style="display:flex; gap:10px; align-items:center; margin:4px 0;">
    <span style="width:12px;height:12px;background:#66bb6a;border-radius:50%;display:inline-block;"></span> Mito/Metabolism
  </div>
  <div style="display:flex; gap:10px; align-items:center; margin:4px 0;">
    <span style="width:12px;height:12px;background:#9aa0a6;border-radius:50%;display:inline-block;"></span> Other
  </div>
</div>
"""

search = """
<div id="kg-search" style="position:fixed; left:12px; top:12px; z-index:9999; background:#fff; border:1px solid #ddd; border-radius:10px; box-shadow:0 2px 8px rgba(0,0,0,0.08); padding:8px 10px;">
  <input id="kg-q" type="text" placeholder="Find node…" style="width:220px; padding:6px 8px; border:1px solid #ccc; border-radius:6px; outline:none;">
</div>
<script>
(function(){
  const input = document.getElementById('kg-q');
  function jump() {
    if (!input || !input.value || typeof network === 'undefined') return;
    const q = input.value.toLowerCase();
    const hits = nodes.get().filter(n => (n.label||'').toLowerCase().includes(q));
    if (!hits.length) return;
    const ids = hits.map(h => h.id);
    network.selectNodes(ids, false);
    network.fit({ nodes: ids, animation: true, maxZoomLevel: 1.2 });
  }
  if (input) {
    input.addEventListener('keydown', (e) => { if (e.key === 'Enter') jump(); });
  }
})();
</script>
"""

# inject legend + search UI before closing body
html_doc = html_doc.replace("</body>", legend + search + "</body>")

# show inline and save
display(HTML(html_doc))
with open(HTML_OUT, "w", encoding="utf-8") as f:
    f.write(html_doc)
print("▶ Saved interactive KG:", HTML_OUT)


In [17]:
# ==========================================
# Enhanced KG Explorer (colorful, searchable)
# Requires: G (analysis graph), G_safe (exportable), first_labels
# ==========================================
%pip -q install python-louvain

import math, colorsys, re, networkx as nx
from pyvis.network import Network
from community import community_louvain  # Louvain modularity
from IPython.display import HTML, display

# -------- Pick which graph to browse --------
# Use the pruned analysis graph for clarity; switch to G_safe for full.
BROWSE_G = G  # or G_safe

# -------- Node communities → distinct colors --------
UG = BROWSE_G.to_undirected()
partition = community_louvain.best_partition(UG, resolution=1.0, random_state=42)
com_ids = sorted(set(partition.values()))
# spread communities around the color wheel
def community_color(cid, total):
    h = (cid / max(1,total)) % 1.0
    r,g,b = colorsys.hsv_to_rgb(h, 0.65, 0.92)
    return f"#{int(r*255):02x}{int(g*255):02x}{int(b*255):02x}"
COMM_COLORS = {cid: community_color(i, max(1,len(com_ids))) for i,cid in enumerate(com_ids)}

# -------- Sizes by weighted degree; clean label helper --------
deg = dict(BROWSE_G.degree(weight="weight"))
in_deg  = dict(BROWSE_G.in_degree(weight="weight")) if BROWSE_G.is_directed() else deg
out_deg = dict(BROWSE_G.out_degree(weight="weight")) if BROWSE_G.is_directed() else deg
def sz(d): return max(10, 10 + 6*math.log1p(d or 1))
def nice_label(n): return BROWSE_G.nodes[n].get("label", n)

# -------- Edge color by relation polarity; width by probability --------
ACT_PAT = re.compile(r"(activat|induc|upregulat|promot|increase|enhanc|stimulat)", re.I)
INH_PAT = re.compile(r"(inhibit|block|suppress|downregulat|decreas|attenuat)", re.I)
def polarity(rel):
    if not rel: return 0
    if ACT_PAT.search(rel): return +1
    if INH_PAT.search(rel): return -1
    return 0
def edge_color(rel):
    pol = polarity(rel or "")
    return "#2e7d32" if pol>0 else ("#c62828" if pol<0 else "#9aa0a6")

# -------- Build PyVis with rich options + search box --------
net = Network(height="800px", width="100%", directed=BROWSE_G.is_directed(), notebook=True, cdn_resources="in_line")

# nicer default physics + UI
import json

net.set_options(json.dumps({
  "interaction": { "hover": True, "navigationButtons": True, "tooltipDelay": 120 },
  "nodes": { "shape": "dot", "scaling": {"min": 6, "max": 40}, "font": {"size": 16} },
  "edges": { "arrows": { "to": { "enabled": True, "scaleFactor": 0.6 } }, "smooth": False },
  "physics": {
    "enabled": True,
    "stabilization": { "enabled": True, "iterations": 300 },
    "barnesHut": { "gravitationalConstant": -20000, "centralGravity": 0.15,
                   "springLength": 160, "springConstant": 0.01, "damping": 0.85 }
  },
  "configure": { "enabled": True, "filter": ["physics","interaction","layout","edges","nodes"] }
}))


# Legend + search injected via HTML template later
def pubmed_url(q):
    from urllib.parse import quote
    return "https://pubmed.ncbi.nlm.nih.gov/?term=" + quote(q)

# pre-compute neighbor summary for tooltips
def neighbor_snippet(n, k=6):
    nbrs = sorted(BROWSE_G[n], key=lambda x: -float(BROWSE_G[n][x].get("p",0.5)))[:k]
    return ", ".join(nice_label(m) for m in nbrs) if nbrs else "—"

for n, data in BROWSE_G.nodes(data=True):
    cid = partition.get(n, 0)
    color = COMM_COLORS.get(cid, "#7e57c2")
    label = nice_label(n)
    deg_w = deg.get(n,0)
    inbound = in_deg.get(n,0); outbound = out_deg.get(n,0)
    tooltip = (
        f"<b>{label}</b><br>"
        f"Community: {cid} &middot; Degree(w): {deg_w}<br>"
        f"In(w): {inbound} &middot; Out(w): {outbound}<br>"
        f"Top neighbors: {neighbor_snippet(n)}<br>"
        f'<a href="{pubmed_url(label)}" target="_blank">Search PubMed</a>'
    )
    net.add_node(
        n,
        label=label,
        title=tooltip,
        color=color,
        value=sz(deg_w)
    )

for u, v, d in BROWSE_G.edges(data=True):
    p = float(d.get("p", 0.5))
    rel = d.get("relation_best", "")
    title = (rel or "related_to") + f" &middot; p={p:.3f} &middot; count={int(d.get('weight',1))}"
    net.add_edge(
        u, v,
        title=title,
        width=max(1, 1 + 6*p),          # thicker if more probable
        color=edge_color(rel),
        arrows="to" if BROWSE_G.is_directed() else ""
    )

# Render HTML and inject a tiny search bar + legend
html = net.generate_html().replace(
    "</body>",
    """
<div style="position:fixed; right:16px; top:16px; z-index:999999; background:#ffffffee; border:1px solid #ddd; border-radius:12px; padding:10px 12px; font-family:Inter,system-ui,-apple-system; box-shadow:0 8px 30px rgba(0,0,0,.08);">
  <div style="font-weight:600; margin-bottom:6px;">KG Explorer</div>
  <input id="nodeSearch" type="text" placeholder="Find node…" style="width:240px; padding:6px 8px; border:1px solid #ccc; border-radius:8px;">
  <button id="goBtn" style="margin-left:6px; padding:6px 10px; border-radius:8px; border:1px solid #bbb; background:#f6f6f6; cursor:pointer;">Go</button>
  <div style="font-size:12px; color:#444; margin-top:8px;">
    <b>Edge colors:</b>
    <span style="color:#2e7d32;">●</span> activation
    <span style="color:#c62828; margin-left:8px;">●</span> inhibition
    <span style="color:#9aa0a6; margin-left:8px;">●</span> neutral/other
  </div>
</div>
<script>
// Node search by label (case-insensitive contains)
(function(){
  const input = document.getElementById('nodeSearch');
  const btn   = document.getElementById('goBtn');
  function focusNode(){
    const q = (input.value || '').toLowerCase();
    if(!q) return;
    const nodes = network.body.data.nodes.get();
    const match = nodes.find(n => (n.label || '').toLowerCase().includes(q));
    if(!match) return;
    network.selectNodes([match.id], false);
    network.focus(match.id, {scale:1.2, animation:{duration:600, easingFunction:'easeInOutQuad'}});
    network.fit({nodes:[match.id], animation:true});
  }
  input.addEventListener('keydown', (e)=>{ if(e.key==='Enter'){ focusNode(); }});
  btn.addEventListener('click', focusNode);
})();
</script>
</body>
"""
)

HTML_OUT_PLUS = os.path.join(OUT, "kg_explorer.html")
with open(HTML_OUT_PLUS, "w", encoding="utf-8") as f:
    f.write(html)

display(HTML(html))
print("✨ Interactive explorer saved to:", HTML_OUT_PLUS)
print("Tip: use the right-side buttons (Physics/Layout) to tweak the layout; hover for rich tooltips.")


✨ Interactive explorer saved to: /content/kg_out/out/kg_explorer.html
Tip: use the right-side buttons (Physics/Layout) to tweak the layout; hover for rich tooltips.
